In [1]:
import pandas as pd
import numpy as np
import re
import time

import sqlalchemy
from sqlalchemy import create_engine
import io
import psycopg2

from pathlib import Path
from collections import Counter

In [2]:
# File imports
# metadata_path = "Data/movies_metadata_clean.csv"
ratings_path = "Data/ratings_clean.csv"
# cast_path = "Data/cast_clean.csv"
# crew_path = "Data/crew_clean.csv"
movies_path = "Data/movies_clean.csv"
genres_path = "New_Data/movie_genres_clean.csv"

In [3]:
# meta_df = pd.read_csv(metadata_path, low_memory=False)
ratings_df = pd.read_csv(ratings_path, low_memory=False)
# cast_df = pd.read_csv(cast_path, low_memory=False)
# crew_df = pd.read_csv(crew_path, low_memory=False)
movies_df = pd.read_csv(movies_path, low_memory=False)
genres_df = pd.read_csv(genres_path, low_memory=False)

In [4]:
# meta_df = meta_df.drop(columns="Unnamed: 0")
# meta_df.head()

In [5]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [6]:
# cast_df = cast_df.drop(columns="Unnamed: 0")
# cast_df.head()

In [7]:
# crew_df = crew_df.drop(columns="Unnamed: 0")
# crew_df.head()

In [8]:
movies_df.head()

,movieId,Title,Year,Adventure,Action,Animation,Children,Comedy,Crime,Documentary,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,No Listed Genre
0,1,Toy Story,1995.0,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995.0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,1995.0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,4,Waiting to Exhale,1995.0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,5,Father of the Bride Part II,1995.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# meta_df.count()

In [10]:
ratings_df.count()

userId       27753444
movieId      27753444
rating       27753444
timestamp    27753444
dtype: int64

In [11]:
# cast_df.count()

In [12]:
# crew_df.count()

In [13]:
movies_df.count()

movieId            58098
Title              57769
Year               57769
Adventure          58098
Action             58098
Animation          58098
Children           58098
Comedy             58098
Crime              58098
Documentary        58098
Drama              58098
Fantasy            58098
Film-Noir          58098
Horror             58098
IMAX               58098
Musical            58098
Mystery            58098
Romance            58098
Sci-Fi             58098
Thriller           58098
War                58098
Western            58098
No Listed Genre    58098
dtype: int64

In [14]:
genres_df.count()

mg_id       106104
genre_id    106104
movie_id    106104
movieId      58098
dtype: int64

In [15]:
# meta_df = meta_df.drop_duplicates(keep="first")
ratings_df = ratings_df.drop_duplicates(keep="first")
# cast_df = cast_df.drop_duplicates(keep="first")
# crew_df = crew_df.drop_duplicates(keep="first")
movies_df = movies_df.drop_duplicates(keep="first")
genres_df = genres_df.drop_duplicates(keep="first")

In [16]:
# meta_df = meta_df.dropna()
ratings_df = ratings_df.dropna()
# cast_df = cast_df.dropna()
# crew_df = crew_df.dropna()
movies_df = movies_df.dropna()
genres_df = genres_df.dropna()

In [17]:
# meta_df.count()

In [18]:
ratings_df.count()

userId       27753444
movieId      27753444
rating       27753444
timestamp    27753444
dtype: int64

In [19]:
# cast_df.count()

In [20]:
# crew_df.count()

In [21]:
movies_df["year"] = movies_df["Year"]
movies_df.count()

movieId            57769
Title              57769
Year               57769
Adventure          57769
Action             57769
Animation          57769
Children           57769
Comedy             57769
Crime              57769
Documentary        57769
Drama              57769
Fantasy            57769
Film-Noir          57769
Horror             57769
IMAX               57769
Musical            57769
Mystery            57769
Romance            57769
Sci-Fi             57769
Thriller           57769
War                57769
Western            57769
No Listed Genre    57769
year               57769
dtype: int64

In [22]:
genres_df.count()

mg_id       58098
genre_id    58098
movie_id    58098
movieId     58098
dtype: int64

In [23]:
movies_df["movie_id"] = movies_df["movieId"]
movies_df["name"] = movies_df["Title"]
movies_df = movies_df.astype({"year": int})
movies_df.head()

,movieId,Title,Year,Adventure,Action,Animation,Children,Comedy,Crime,Documentary,...,Mystery,Romance,Sci-Fi,Thriller,War,Western,No Listed Genre,year,movie_id,name
0,1,Toy Story,1995.0,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,1995,1,Toy Story
1,2,Jumanji,1995.0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1995,2,Jumanji
2,3,Grumpier Old Men,1995.0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1995,3,Grumpier Old Men
3,4,Waiting to Exhale,1995.0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1995,4,Waiting to Exhale
4,5,Father of the Bride Part II,1995.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1995,5,Father of the Bride Part II


In [24]:
ratings_df["user_id"] = ratings_df["userId"]
ratings_df["movie_id"] = ratings_df["movieId"]
ratings_df.head()

,userId,movieId,rating,timestamp,user_id,movie_id
0,1,307,3.5,1256677221,1,307
1,1,481,3.5,1256677456,1,481
2,1,1091,1.5,1256677471,1,1091
3,1,1257,4.5,1256677460,1,1257
4,1,1449,4.5,1256677264,1,1449


In [25]:
genres_df.head(20)

,mg_id,genre_id,movie_id,movieId
0,0,1,1,1.0
1,1,3,1,2.0
2,2,4,1,3.0
3,3,5,1,4.0
4,4,9,1,5.0
5,5,1,2,6.0
6,6,4,2,7.0
7,7,9,2,8.0
8,8,5,3,9.0
9,9,15,3,10.0


In [26]:
genres_df = genres_df.loc[:, ["mg_id", "movie_id", "genre_id"]]
movies_df = movies_df.loc[:, ["movie_id", "name", "year"]]
ratings_df = ratings_df.loc[:, ["user_id", "movie_id", "rating"]]

genres_df = genres_df.set_index("mg_id")
movies_df = movies_df.set_index("movie_id")
ratings_df = ratings_df.set_index("user_id")

In [27]:
genres_df.head()

,movie_id,genre_id
mg_id,,
0,1,1
1,1,3
2,1,4
3,1,5
4,1,9


In [28]:
movies_df.head()

,name,year
movie_id,,
1,Toy Story,1995
2,Jumanji,1995
3,Grumpier Old Men,1995
4,Waiting to Exhale,1995
5,Father of the Bride Part II,1995


In [29]:
ratings_df.head()

,movie_id,rating
user_id,,
1,307,3.5
1,481,3.5
1,1091,1.5
1,1257,4.5
1,1449,4.5


In [30]:
# meta_df.to_csv("New_Data/movie_metadata_clean.csv", index=False)
# ratings_df.to_csv("New_Data/ratings_clean.csv", index=False)
# cast_df.to_csv("New_Data/cast_clean.csv", index=False)
# crew_df.to_csv("New_Data/crew_clean.csv", index=False)
# movies_df.to_csv("New_Data/movies_clean.csv", index=False)

In [31]:
from config import db_password
from config import user

In [43]:
host = 'movies-fp.cpige012zhtw.us-east-1.rds.amazonaws.com'
port = 5432
passw = db_password
database = "postgres"
port=5432

   
db_string = "postgresql://" + user + ":" + passw + "@" + host + ":" + "5432/" + database
engine = create_engine(db_string)

In [45]:
genres_df.to_sql(name="movie_genre", con=engine, if_exists="append")

In [46]:
movies_df.to_sql(name="movies", con=engine, if_exists="append")

In [47]:
ratings_df.to_sql(name="ratings", con=engine, if_exists="append")